# LIMEを用いたMNIST画像分類

In [3]:
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers, models, initializers
from tensorflow.keras.datasets import mnist

np.random.seed(1)
tf.random.set_seed(1)

print("tensorflow version : " + tf.__version__)

ModuleNotFoundError: No module named 'tensorflow'